In [1]:
import json
import numpy as np
import pandas as pd

def tanh(x): return np.tanh(x)
def linear(x): return x
ACT = {"tanh": tanh, "linear": linear}

# ---- SAME FEATURES ----
def make_features(df):
    s  = df["Strain"].astype(float).values
    sr = df["Strain rate (s-1)"].astype(float).values
    T  = df["Deformation Temperature"].astype(float).values

    X = pd.DataFrame({
        "s": s,
        "sr": sr,
        "T": T,
        "s2": s**2,
        "s3": s**3,
        "sr2": sr**2,
        "sr3": sr**3,
        "s_sr": s * sr,
        "s_T": s * T,
        "sr_T": sr * T,
        "log_s": np.log1p(np.abs(s)),
        "log_sr": np.log1p(sr),
        "invT": 1/(T + 273.15),
        "exp_invT": np.exp(1/(T + 273.15)),
        "s_over_sr": s/(sr+1e-8),
        "sr_over_s": sr/(s+1e-8)
    })
    return X.values


def manual_forward(x, M):
    x_min = np.array(M["x_min"], dtype=float)
    x_max = np.array(M["x_max"], dtype=float)
    y_min = float(M["y_min"])
    y_max = float(M["y_max"])

    x_scaled = (x - x_min) / (x_max - x_min)

    a = x_scaled
    for layer in M["layers"]:
        W = np.array(layer["weights"], dtype=float)
        b = np.array(layer["bias"], dtype=float)
        act = ACT[layer["activation"]]
        a = act(np.dot(a, W) + b)

    y_scaled = a[0]
    y_real = y_scaled * (y_max - y_min) + y_min
    return float(y_real)


def main():
    with open("ANN_Matrix.json") as f:
        M = json.load(f)

    df = pd.read_csv("Training Data.csv").dropna()
    X = make_features(df)

    print("\nChecking first 10 samples using matrix:\n")

    for i in range(10):
        pred = manual_forward(X[i], M)
        print(f"Sample {i}: Predicted Flow Stress = {pred:.3f}")

    print("\nMatrix forward pass OK.\n")


if __name__ == "__main__":
    main()



Checking first 10 samples using matrix:

Sample 0: Predicted Flow Stress = 40.932
Sample 1: Predicted Flow Stress = 32.750
Sample 2: Predicted Flow Stress = 22.955
Sample 3: Predicted Flow Stress = 56.955
Sample 4: Predicted Flow Stress = 68.706
Sample 5: Predicted Flow Stress = 162.779
Sample 6: Predicted Flow Stress = 143.008
Sample 7: Predicted Flow Stress = 98.206
Sample 8: Predicted Flow Stress = 38.079
Sample 9: Predicted Flow Stress = 31.925

Matrix forward pass OK.

